## 네이버 뉴스 크롤링 최종
- 섹션 추출~ sub 페이지 추출
#### (1) + (2) + (3)
(1) 섹션 메뉴와 섹션별 url 추출      
    - section, link  
    - section_menu_df  
(2) 섹션별 헤드라인(topic) 제목과 url 추출  
    - topic, url, section  
    - all_topic_df  
(3) 섹션 헤드라인별 sub 페이지 기사 내용 추출  
    - 언론사, 제목, 날짜/시간, 기사내용, 작성자  
    - paper, title, datetime, writing, writer  
    - all_sub_news_info_df   

In [1]:
from urllib.request import urlopen
import pandas as pd
import bs4
import requests

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [3]:
url = 'https://news.naver.com/'
html = urlopen(url)
bs_obj = bs4.BeautifulSoup(html, 'html.parser')
# bs_obj

### (1) 네이버 뉴스 섹션 메뉴, url추출

In [4]:
ul = bs_obj.find('ul', {'class':'Nlnb_menu_list'})
# ul

In [5]:
# 수집된 데이터로 df구성
lis = ul.select('li')

section = []
link = []

# 리스트에 추가
for li in lis:
    a_tag = li.find('a')
    section.append(a_tag.text)
    link.append(a_tag['href'])

section_menu_df = pd.DataFrame({'section':section, 'link':link})
section_menu_df

,section,link
0,언론사별,https://news.naver.com/?viewType=pc
1,정치,https://news.naver.com/main/main.naver?mode=LS...
2,경제,https://news.naver.com/main/main.naver?mode=LS...
3,사회,https://news.naver.com/main/main.naver?mode=LS...
4,생활/문화,https://news.naver.com/main/main.naver?mode=LS...
5,IT/과학,https://news.naver.com/main/main.naver?mode=LS...
6,세계,https://news.naver.com/main/main.naver?mode=LS...
7,랭킹,https://news.naver.com/main/ranking/popularDay...
8,신문보기,https://news.naver.com/newspaper/home?viewType=pc
9,오피니언,https://news.naver.com/opinion/home


### (2) 섹션별 헤드라인(topic) 제목, url 추출

In [6]:
# 함수로 구성
# 섹션과 url 전달 받아서, 헤드라인 topic의 제목, 링크, 섹션 추출하여
# 딕셔너리로 만들어 반환하는 함수
def get_topic(url, section):
    # 접속 및 결과 파싱
    headers = {'User-Agent':'Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1'}
    res = requests.get(url, headers=headers)
    html = res.text
    bs_obj = bs4.BeautifulSoup(html, 'html.parser')

    # topic_list 추출
    topic_list = bs_obj.findAll('div', {'class':'sh_text'})


    # 제목, 링크 추출해서 리스트에 넣기
    topic_title = []
    topic_link = []

    for topic in topic_list:
        try:
            a = topic.find('a')
            topic_title.append(a.text)
            topic_link.append(a['href'])
        except:
            print('오류발생')

    # 딕셔너리 만들어서 반환
    return ({'topic':topic_title, 'url':topic_link, 'section':section})

In [7]:
# 모든 섹션에 대한 모든 topic 데이터프레임 생성
# 빈 데이터프레임 만들고
all_topic_df = pd.DataFrame({
    'topic' : [],
    'url':[],
    'section':[]
})

for i in range(1,7):
    df = pd.DataFrame(get_topic(section_menu_df['link'][i], section_menu_df['section'][i]))
    all_topic_df = pd.concat([all_topic_df,df], axis=0, ignore_index=True)
# all_topic_df

#### (3) 섹션 헤드라인별 Sub페이지 기사 내용 추출
- 언론사, 제목, 날짜/시간, 기사내용, 작성자

In [8]:
# sub 페이지 추출 함수
def get_sub_news_info(url):
    headers = {'User-Agent':'Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1'}
    # 빈 리스트 생성
    paper_list, title_list, datetime_list, writing_list, writer_list = [ [] for _ in range(5) ]
    # for _ in range(5) : _ 언더바 - 변수 없이 반복만 수행
    
    try:
        # 접속
        res = requests.get(url, headers = headers)
        html = res.text
        bs_obj = bs4.BeautifulSoup(html, 'html.parser')

        # 언론사, 기사제목, 작성 날짜 시간, 기사 내용, 작성자 리스트 생성
        paper_list.append(bs_obj.find('a', {'class':'media_end_head_top_logo'}).select_one('img')['title'])
        title_list.append(bs_obj.find('div', {'class':'media_end_head_title'}).select_one('span').text)
        datetime_list.append(bs_obj.find('span', {'class':'media_end_head_info_datestamp_time _ARTICLE_DATE_TIME'}))
        writing_list.append(bs_obj.select_one('#dic_area').text.replace('\n',''))
        writer_list.append(bs_obj.find('span',{'class':'byline_s'}).text)

    except:
        print('sub_news error occurred!')
        

    # dictionary로 만들어서 반환
    sub_news_dict = {
        'paper' : paper_list,
        'title' : title_list,
        'datetime' : datetime_list,
        'writing' : writing_list,
        'writer' : writer_list
    }
    return sub_news_dict


In [9]:
# 빈 데이터 프레임 생성
all_sub_news_info_df = pd.DataFrame({
    'paper' : [],
    'title' : [],
    'datetime' : [],
    'writing' : [],
    'writer' : []
})

# all_topic_df에 있는 모든 서브 페이지 url전달하고 결과 받아서 최종 데이터프레임으로 저장
for url in all_topic_df['url']:
    df = pd.DataFrame(get_sub_news_info(url))
    all_sub_news_info_df = pd.concat([all_sub_news_info_df, df], axis=0, ignore_index=True)
all_sub_news_info_df

sub_news error occurred!


ValueError: All arrays must be of the same length

In [ ]:
# 최종 결과 파일로 저장
all_sub_news_info_df.to_csv('./crawl_data/naver_sub_news_section.csv', index=0)

In [ ]:
all_sub_news_info_df = pd.read_csv('./crawl_data/naver_sub_news_section.csv')
all_sub_news_info_df